In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
import duckdb
# Assessment Question 1 - Of the industries where salaries and wages data did NOT exist in 2016 and only appeared later,
# which industry had the highest average value for actual filled jobs across time and what was that value.

result = duckdb.execute("""
with post_2016 as
(
    select
        distinct Series_title_2 as industry
    from 'financial_data.csv'
    where
        lower(Series_title_1) = 'salaries and wages'
        and cast(substr(cast(Period as string),1,4) as integer) >2016
        and Series_title_2 not in
                            (
                            select
                              Series_title_2
                            from 'financial_data.csv'
                            where
                                lower(Series_title_1) = 'salaries and wages'
                                and cast(substr(cast(Period as string),1,4) as integer) = 2016
                            )
)
,industry_filled_jobs as
(
  select
      Series_title_2 AS industry,
      AVG(Data_value) AS avg_filled_jobs
    from 'employment_data.csv'
    where
        "Group" = 'Industry by employment variable'
        and Series_title_1 = 'Filled jobs'
        and Series_title_3 = 'Actual'
        and Series_title_2 IN (select industry from post_2016)
    group by Series_title_2
)

select  * from industry_filled_jobs order  by avg_filled_jobs desc limit 1""")
df = result.fetchdf()
print(df)






       industry  avg_filled_jobs
0  Retail Trade    194053.711538


In [ ]:
# Assessment Question 2 - Query to show which year/month combination and industry had the second highest seasonally adjusted operating income sales across all the
#business industries in New Zealand that are categorised as NZSIOC level 2.

result = duckdb.execute("""
with ranked_sales as
(
	select
	    Period
	    ,Series_title_2 as industry
	    ,Data_value as operating_income_sales
	    ,row_number() over (order by Data_value desc) as sales_rank
  from
      'financial_data.csv'
  where
    Series_title_1	= 'Sales (operating income)'
    and lower(Series_title_4) = 'seasonally adjusted'
    and "Group" = 'Industry by financial variable (NZSIOC Level 2)'
)

select
    Period
    ,industry
    ,operating_income_sales
from ranked_sales
where sales_rank = 2""")
df = result.fetchdf()
print(df)


    Period         industry  operating_income_sales
0  2023.03  Wholesale Trade               38810.022


In [ ]:
#Assessment Question- 3  Query to calculate the quarterly cumulative number of filled jobs over time for the territorial authority with the highest average value of filled jobs across time.
#Note: As per the assessment instruction, windows functions are not used in the below query.

result = duckdb.execute("""
with avg_filled_jobs as
(
  select
      Series_title_2 AS territorial_authority
      ,AVG(Data_value) AS avg_filled_jobs
  from 'employment_data.csv'
  where
      "Group" = 'Territorial authority by employment variable'
      and Series_title_1 = 'Filled jobs'
      and Series_title_3 = 'Actual'
  group by Series_title_2
)

,top_ta as
(
  select
      territorial_authority
  from
      avg_filled_jobs
  order by avg_filled_jobs desc
  limit 1
)

,filled_jobs_filtered as (
    select *
    from 'employment_data.csv'
    where
        "Group" = 'Territorial authority by employment variable'
        and Series_title_1 = 'Filled jobs'
        and Series_title_3 = 'Actual'
        and Series_title_2 = (select territorial_authority from top_ta)
)

select
    a.Period,
    a.Series_title_2 AS territorial_authority,
    a.Data_value AS filled_jobs,
    sum(b.Data_value) AS cumulative_filled_jobs
from filled_jobs_filtered a
join filled_jobs_filtered b
  on b.Period <= a.Period
group by a.Period, a.Series_title_2, a.Data_value
order by a.Period""")
df = result.fetchdf()
print(df)





     Period territorial_authority  filled_jobs  cumulative_filled_jobs
0   2011.06              Auckland     558627.0                558627.0
1   2011.09              Auckland     565439.0               1124066.0
2   2011.12              Auckland     573729.0               1697795.0
3   2012.03              Auckland     560373.0               2258168.0
4   2012.06              Auckland     572938.0               2831106.0
5   2012.09              Auckland     570677.0               3401783.0
6   2012.12              Auckland     579724.0               3981507.0
7   2013.03              Auckland     574807.0               4556314.0
8   2013.06              Auckland     588911.0               5145225.0
9   2013.09              Auckland     586356.0               5731581.0
10  2013.12              Auckland     598580.0               6330161.0
11  2014.03              Auckland     591999.0               6922160.0
12  2014.06              Auckland     604461.0               7526621.0
13  20

#Assessment Question - 4
#Assume these datasets are used in part of a pipeline where the file that arrives may contain unwanted duplicates, incorrect datatypes, missing dates or other data quality aberrations. What things could be done programmatically to make sure the input data is of adequate quality and improve the pipeline?


1. For removing duplicates, select distinct * or column name can be used
    e.g. df.drop_duplicates(inplace=True)

2. For incorrect data types, columns can be casted to appropriate data types using cast function.
   e.g. cast(Data_Value as double)
   In my code I have used cast function to convert period column to string.
    e.g. CAST(substr(cast(Period as string),1,4) AS INTEGER) = 2016
3. Identify null  or blank fields and handle missing values.
    e.g. df.isnull().sum()

4.  Data formats - Ensure date fields (Period) follows proper format (YYYY.MM in this case) in all rows.

5. handle outliers or data abnornamlities.
    Apply conditions like Data_value >=0 or define value ranges.
    e.g. df = df[df['year'] <= 2024]
    e.g.Data_value >=0


In [21]:
#Assessment Question-5 Create summary statistics and perform a statistical analysis or create a model using the provided datasets.

import pandas as pd
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt

#loading files
emp_df = pd.read_csv('employment_data.csv')
fin_df = pd.read_csv('financial_data.csv')

# printing data
print("Employment data:")
display(emp_df.head())

print("Financial data:")
display(fin_df.head())

# Understanding the data by checking shapes and missing values
print("\nEmployment shape:", emp_df.shape)
print("\nFinancial shape:", fin_df.shape)

print("\nMissing values (employment):")
display(emp_df.isnull().sum())

print("\nMissing values (financial):")
display(fin_df.isnull().sum())


# Correcting the filter condition for employment data
#emp_filtered = emp_df[emp_df['Series_title_1'] == 'Filled jobs'] #Changed to match Series_title_1 in emp_df
# Correcting the filter condition for financial data and selecting relevant column
#fin_filtered = fin_df[fin_df['Series_title_1'] == 'Salaries and wages paid'] #Changed to match Series_title_1 in fin_df


#merging dataframes on Period and Group
#merged_df = pd.merge(emp_filtered, fin_filtered, on=['Period', 'Group'], suffixes=('_jobs', '_wages'))


#Renaming the columns to 'value_jobs' and 'value_wages'
#merged_df = merged_df.rename(columns={'Data_value_jobs': 'value_jobs', 'Data_value_wages': 'value_wages'}) #Renamed Data_value columns as value_jobs and value_wages


#X = merged_df[['value_jobs']]  # independent variable (filled jobs)
#y = merged_df['value_wages']   # dependent variable (wages)


#X = X.astype(float)
#y = y.astype(float)


#model = LinearRegression()
#model.fit(X, y)

#plt.scatter(X, y, alpha=0.5)





#Linear Regression model can be used for predicting future values. For e.g. predicting employment based on financial data





Employment data:


,Series_reference,Period,Data_value,Suppressed,STATUS,UNITS,Magnitude,Subject,Group,Series_title_1,Series_title_2,Series_title_3,Series_title_4,Series_title_5
0,BDCQ.SEA1AA,2011.06,80078.0,NaN,F,Number,0,Business Data Collection - BDC,Industry by employment variable,Filled jobs,"Agriculture, Forestry and Fishing",Actual,NaN,NaN
1,BDCQ.SEA1AA,2011.09,78324.0,NaN,F,Number,0,Business Data Collection - BDC,Industry by employment variable,Filled jobs,"Agriculture, Forestry and Fishing",Actual,NaN,NaN
2,BDCQ.SEA1AA,2011.12,85850.0,NaN,F,Number,0,Business Data Collection - BDC,Industry by employment variable,Filled jobs,"Agriculture, Forestry and Fishing",Actual,NaN,NaN
3,BDCQ.SEA1AA,2012.03,90743.0,NaN,F,Number,0,Business Data Collection - BDC,Industry by employment variable,Filled jobs,"Agriculture, Forestry and Fishing",Actual,NaN,NaN
4,BDCQ.SEA1AA,2012.06,81780.0,NaN,F,Number,0,Business Data Collection - BDC,Industry by employment variable,Filled jobs,"Agriculture, Forestry and Fishing",Actual,NaN,NaN


Financial data:


,Series_reference,Period,Data_value,Suppressed,STATUS,UNITS,Magnitude,Subject,Group,Series_title_1,Series_title_2,Series_title_3,Series_title_4,Series_title_5
0,BDCQ.SF1AA2CA,2016.06,1116.386,NaN,F,Dollars,6,Business Data Collection - BDC,Industry by financial variable (NZSIOC Level 2),Sales (operating income),Forestry and Logging,Current prices,Unadjusted,NaN
1,BDCQ.SF1AA2CA,2016.09,1070.874,NaN,F,Dollars,6,Business Data Collection - BDC,Industry by financial variable (NZSIOC Level 2),Sales (operating income),Forestry and Logging,Current prices,Unadjusted,NaN
2,BDCQ.SF1AA2CA,2016.12,1054.408,NaN,F,Dollars,6,Business Data Collection - BDC,Industry by financial variable (NZSIOC Level 2),Sales (operating income),Forestry and Logging,Current prices,Unadjusted,NaN
3,BDCQ.SF1AA2CA,2017.03,1010.665,NaN,F,Dollars,6,Business Data Collection - BDC,Industry by financial variable (NZSIOC Level 2),Sales (operating income),Forestry and Logging,Current prices,Unadjusted,NaN
4,BDCQ.SF1AA2CA,2017.06,1233.700,NaN,F,Dollars,6,Business Data Collection - BDC,Industry by financial variable (NZSIOC Level 2),Sales (operating income),Forestry and Logging,Current prices,Unadjusted,NaN



Employment shape: (22963, 14)

Financial shape: (7635, 14)

Missing values (employment):


,0
Series_reference,0
Period,0
Data_value,2839
Suppressed,20124
STATUS,0
UNITS,0
Magnitude,0
Subject,0
Group,0
Series_title_1,0



Missing values (financial):


,0
Series_reference,0
Period,0
Data_value,698
Suppressed,7620
STATUS,0
UNITS,0
Magnitude,0
Subject,0
Group,0
Series_title_1,0
